In [1]:
import numpy as np

In [2]:
def evaluation_forward(A, B, PI, O, alpha):
    alpha[:, 0] = np.multiply(PI, B[:, O[0]])
    for t in range(1, O.shape[0]):
        for j in range(A.shape[0]):
            alpha[j, t] = np.sum(np.multiply(alpha[:, t - 1], A[:, j])) * B[j, O[t]]
    return alpha

In [3]:
def evaluation_backward(A, B, PI, O, beta):
    beta[:, -1] = np.ones((A.shape[0]), dtype=np.float64)
    for t in range(O.shape[0] - 2, -1, -1):
        for i in range(A.shape[0]):
            beta[i, t] = np.sum(np.multiply(np.multiply(A[i, :], B[:, O[t]]), beta[:, t + 1]))
    return beta

In [4]:
def decode(A, B, PI, O, delta):
    delta[:, 0] = np.multiply(PI, B[:, O[0]])
    for t in range(1, O.shape[0]):
        for j in range(A.shape[0]):
            delta[j, t] = np.amax(np.multiply(delta[:, t - 1], A[:, j])) * B[j, O[t]]
    path = np.argmax(delta, axis=0) + 1
    return delta, path

In [5]:
def learn(A, B, PI, O, V, alpha, beta, XI, gamma):
    alpha = evaluation_forward(A, B, PI, O, alpha)
    beta = evaluation_backward(A, B, PI, O, beta)
    
    #Compute XI#
    p = np.sum(np.multiply(alpha, beta), axis=0)
    for t in range(O.shape[0] - 1):
        for i in range(A.shape[0]):
            for j in range(A.shape[0]):
                XI[i, j, t] = (alpha[i, t] * beta[j, t + 1] * A[i, j] * B[j, O[t + 1]]) / p[t]
    
    #Compute gamma#
    for t in range(O.shape[0]):
        for i in range(A.shape[0]):
            gamma[i, t] = np.sum(XI[i, :, t])
            
    #Compute new A#
    for i in range(A.shape[0]):
        for j in range(B.shape[0]):
            A[i, j] = np.sum(XI[i, j, :]) / np.sum(gamma[i])
    
    #Compute new PI#
    for i in range(A.shape[0]):
        PI[i] = gamma[i, 0]
        
    #Compute new B#
    for j in range(A.shape[0]):
        for k in range(V.shape[0]):
            B[j, k] = np.sum(np.multiply(gamma[j], (O == V[k]).astype(np.float64))) / np.sum(gamma[j])
    return A, B, PI, gamma, XI, alpha, beta

In [6]:
A = np.array([ #1st Rank = From State, 2nd Rank = To State
    [1 / 3, 1 / 3, 1 / 3],
    [1 / 3, 1 / 3, 1 / 3],
    [1 / 3, 1 / 3, 1 / 3]
], dtype=np.float64)
PI = np.array([1/ 3, 1 / 3, 1 / 3], dtype=np.float64) #1st Rank = Init State
B = np.array([ #1st Rankg = State, 2nd Rank = Output Value
    [4 / 6, 2 / 6],
    [2 / 6, 4 / 6],
    [3 / 6, 3 / 6]
], dtype=np.float64)
O = np.array([0, 0, 1, 0, 1], dtype=np.int32) #0 = red, 1 = blue
V = np.array([0, 1], dtype=np.int32) #0 = red, 1 = blue
alpha = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
beta = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
delta = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
gamma = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
XI = np.zeros((B.shape[0], B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = From State, 2nd Rank = To State, 3rd Rank = Time

In [7]:
for i in range(100):
    A, B, PI, gamma, XI, alpha, beta = learn(A, B, PI, O, V, alpha, beta, XI, gamma)
    delta, path = decode(A, B, PI, O, delta)
    print('Path: {}'.format(path), end='')
    if np.isnan(np.linalg.norm(A)) or np.isnan(np.linalg.norm(B)) or np.isnan(np.linalg.norm(PI)):
        print('\tNaN occurred !!', end='')
    print()

Path: [1 1 2 1 2]
Path: [1 1 2 1 2]
Path: [1 1 2 1 2]
Path: [1 1 2 1 2]
Path: [1 3 2 1 2]
Path: [1 1 2 1 2]
Path: [1 3 2 1 2]
Path: [1 1 2 1 2]
Path: [1 3 3 1 2]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 3 3 1 3]
Path: [1 3 3 3 3]
Path: [1 3 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1 3 1 3]
Path: [1 3 3 3 3]
Path: [1 1

/home/lab1323/.local/lib/python3.5/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
/home/lab1323/.local/lib/python3.5/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars
